In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta

# Standard plotly imports
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
init_notebook_mode(connected=True)

In [2]:
data = pd.read_csv('africa_data.csv')

In [3]:
df = data.groupby(['date', 'Country/Region'], as_index=False)['Confirmed'].sum()
df['previous_week'] = df.groupby(['Country/Region'])['Confirmed'].shift(7, fill_value=0)
df['new_cases'] = df['Confirmed'] - df['previous_week']

In [4]:
threshold = 50
countries = df.groupby(by='Country/Region', as_index=False)['Confirmed'].max().sort_values(by='Confirmed', ascending=False)
countries = countries[countries['Confirmed'] > threshold]['Country/Region'].unique()

fig = go.Figure()
for country in countries:
    filtered_df = df[df['Country/Region'] == country].reset_index()
    idx = filtered_df['Confirmed'].sub(threshold).gt(0).idxmax()
    trace_data = filtered_df[idx:]
    fig.add_trace(go.Scatter(x=trace_data['Confirmed'],
                             y=trace_data['new_cases'],
                             mode='lines',
                             name=country,
                             text=df['date'],
                             hoverinfo='x+text+name'))
fig.update_layout(xaxis_type="log",
                  yaxis_type="log",
                  title='Trajectory of COVID-19 Confirmed Cases (countries with greater than 50 confirmed cases)',
                  xaxis_title='Total Confirmed Cases',
                  yaxis_title='New Confirmed Cases (in the past week)')
fig.show()

In [5]:
east = ['Burundi',
 'Comoros',
 'Djibouti',
 'Eritrea',
 'Ethiopia',
 'Kenya',
 'Madagascar',
 'Malawi',
 'Mauritius',
 'Mozambique',
 'Rwanda',
 'Seychelles',
 'Somalia',
 'South Sudan',
 'Tanzania',
 'Uganda',
 'Zambia',
 'Zimbabwe']
df3 = data[data['Country/Region'].isin(east)].groupby(['date', 'Country/Region'], as_index=False)['Confirmed'].sum()
df3['previous_week'] = df3.groupby(['Country/Region'])['Confirmed'].shift(7, fill_value=0)
df3['new_cases'] = df3['Confirmed'] - df3['previous_week']

In [6]:
threshold = 50
countries = df3.groupby(by='Country/Region', as_index=False)['Confirmed'].max().sort_values(by='Confirmed', ascending=False)
countries = countries[countries['Confirmed'] > threshold]['Country/Region'].unique()

fig = go.Figure()
for country in countries:
    filtered_df = df3[df3['Country/Region'] == country].reset_index()
    idx = filtered_df['Confirmed'].sub(threshold).gt(0).idxmax()
    trace_data = filtered_df[idx:]
    fig.add_trace(go.Scatter(x=trace_data['Confirmed'],
                             y=trace_data['new_cases'],
                             mode='lines',
                             name=country,
                             text=df3['date'],
                             hoverinfo='x+text+name'))
fig.update_layout(xaxis_type="log",
                  yaxis_type="log",
                  title='Trajectory of COVID-19 Confirmed Cases in Eastern Africa (countries with greater than 50 confirmed cases)',
                  xaxis_title='Total Confirmed Cases',
                  yaxis_title='New Confirmed Cases (in the past week)')
fig.show()

In [17]:
df.tail()

,date,Country/Region,Confirmed,previous_week,new_cases
880,2020-04-01,Togo,36,23,13
881,2020-04-01,Tunisia,423,173,250
882,2020-04-01,Uganda,44,14,30
883,2020-04-01,Zambia,36,12,24
884,2020-04-01,Zimbabwe,8,3,5


In [19]:
dates = df['date'].unique()
threshold = 10

# make list of countries
countries = df.groupby(by='Country/Region', as_index=False)['Confirmed'].max().sort_values(by='Confirmed', ascending=False)
countries = countries[countries['Confirmed'] > threshold]['Country/Region'].unique()

# make figure
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

# fill in most of layout
fig_dict["layout"]["xaxis"] = {'range': [3, 5.25],
                               'title': 'Total Confirmed Cases',
                               'type': 'log'}
fig_dict["layout"]["yaxis"] = {'range': [2.25, 5.25],
                               'title': 'New Confirmed Cases (in the past week)',
                               'type': 'log'}
fig_dict['layout']['title'] = 'Trajectory of COVID-19 Confirmed Cases (countries with greater than 50 confirmed cases)'
fig_dict["layout"]["hovermode"] = "closest"
fig_dict["layout"]["sliders"] = {
    "args": [
        "transition", {
            "duration": 400,
            "easing": "cubic-in-out"
        }
    ],
    "initialValue": "2020-01-22",
    "plotlycommand": "animate",
    "values": dates,
    "visible": True
}
fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Date:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

# make data
date = '2020-01-22'
for country in countries:
    filtered_df = df[df['Country/Region'] == country].reset_index()
    idx = filtered_df['Confirmed'].sub(threshold).gt(0).idxmax()
    trace_data = filtered_df[idx:]
    
    dataset_by_date = trace_data[trace_data['date'] <= date]
    dataset_by_date_and_country = dataset_by_date[
        dataset_by_date['Country/Region'] == country]

    data_dict = {
        "x": list(dataset_by_date_and_country['Confirmed']),
        "y": list(dataset_by_date_and_country['new_cases']),
        "mode": 'lines',
        "text": list(dataset_by_date_and_country['date']),
        "name": country
    }
    fig_dict["data"].append(data_dict)

# make frames
for date in dates:
    frame = {"data": [], "name": str(date)}
    for country in countries:
        filtered_df = df[df['Country/Region'] == country].reset_index()
        idx = filtered_df['Confirmed'].sub(threshold).gt(0).idxmax()
        trace_data = filtered_df[idx:]
    
        dataset_by_date = trace_data[trace_data["date"] <= date]
        dataset_by_date_and_country = dataset_by_date[
            dataset_by_date['Country/Region'] == country]

        data_dict = {
            "x": list(dataset_by_date_and_country['Confirmed']),
            "y": list(dataset_by_date_and_country['new_cases']),
            "mode": "lines",
            "text": list(dataset_by_date_and_country['date']),
            "name": country
        }
        frame["data"].append(data_dict)

    fig_dict["frames"].append(frame)
    slider_step = {"args": [
        [date],
        {"frame": {"duration": 300, "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 300}}
    ],
        "label": date,
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)


fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

fig.show()


ValueError: attempt to get argmax of an empty sequence